## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-16-19-47-15 +0000,nyt,"Israel Strikes Syria’s Capital, Sending Warnin...",https://www.nytimes.com/2025/07/16/world/middl...
1,2025-07-16-19-43-08 +0000,nyt,Carney Moves to Reduce Canada’s Chinese Steel ...,https://www.nytimes.com/2025/07/16/world/canad...
2,2025-07-16-19-34-10 +0000,startribune,Investigators search Housing Stabilization Ser...,https://www.startribune.com/housing-stabilizat...
3,2025-07-16-19-32-54 +0000,nyt,At Least 20 Killed in Stampede Outside a Gaza ...,https://www.nytimes.com/2025/07/16/world/middl...
4,2025-07-16-19-22-03 +0000,nyt,"As Iran Deports a Million Afghans, ‘Where Do W...",https://www.nytimes.com/2025/07/16/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2317/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
15,trump,9
0,israel,7
34,iran,4
143,pm,4
74,minneapolis,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
1,2025-07-16-19-43-08 +0000,nyt,Carney Moves to Reduce Canada’s Chinese Steel ...,https://www.nytimes.com/2025/07/16/world/canad...,23
24,2025-07-16-17-00-06 +0000,startribune,Trump slams his own supporters as 'weaklings' ...,https://www.startribune.com/trump-slams-his-ow...,23
6,2025-07-16-19-15-59 +0000,bbc,Canada curbs steel imports to shield domestic ...,https://www.bbc.com/news/articles/cy5wv7rzvk0o,22
16,2025-07-16-18-28-47 +0000,bbc,Trump discussed firing Fed boss but 'highly un...,https://www.bbc.com/news/articles/c4geyrdprwjo,20
44,2025-07-16-03-00-00 +0000,world,"Through Trial and Error, Iran Found Gaps in Is...",https://www.wsj.com/world/middle-east/iran-isr...,20


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
1,23,2025-07-16-19-43-08 +0000,nyt,Carney Moves to Reduce Canada’s Chinese Steel ...,https://www.nytimes.com/2025/07/16/world/canad...
44,20,2025-07-16-03-00-00 +0000,world,"Through Trial and Error, Iran Found Gaps in Is...",https://www.wsj.com/world/middle-east/iran-isr...
20,18,2025-07-16-17-22-24 +0000,bbc,Serious questions for Tories over Afghan data ...,https://www.bbc.com/news/articles/c98w2e9leywo
22,14,2025-07-16-17-12-33 +0000,startribune,A $70 kosher pastrami sandwich? That’s what on...,https://www.startribune.com/minneapolis-dfl-co...
24,14,2025-07-16-17-00-06 +0000,startribune,Trump slams his own supporters as 'weaklings' ...,https://www.startribune.com/trump-slams-his-ow...
38,14,2025-07-16-10-35-23 +0000,nyt,South African AIDS Activist Pushes for H.I.V. ...,https://www.nytimes.com/2025/07/14/health/sout...
35,13,2025-07-16-12-20-00 +0000,world,Nvidia CEO Lavishes Praise on China in Beijing...,https://www.wsj.com/world/asia/nvidia-ceo-lavi...
0,11,2025-07-16-19-47-15 +0000,nyt,"Israel Strikes Syria’s Capital, Sending Warnin...",https://www.nytimes.com/2025/07/16/world/middl...
2,11,2025-07-16-19-34-10 +0000,startribune,Investigators search Housing Stabilization Ser...,https://www.startribune.com/housing-stabilizat...
17,11,2025-07-16-18-23-44 +0000,startribune,Vance Boelter plans to plead not guilty in sho...,https://www.startribune.com/vance-boelter-plan...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
